In [1]:
from mpl_toolkits.mplot3d import Axes3D 
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.graph_objs import *
import plotly.express as px

import seaborn as sns
import os
from glob import glob
import string

import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from numpy import linalg as LA
from sklearn.metrics import pairwise_distances_argmin_min
import hdbscan
from scipy.cluster.hierarchy import fcluster
from sklearn import preprocessing
from sklearn.cluster import KMeans
import umap

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Plot WSI with colored labels
def scattered_wsi(df,x,y,hue,size,opacity,auto_open,filename):
    fig = px.scatter(df,
                     x=x, y=y,color=hue,
                     width=800, height=800,
                     color_discrete_sequence=px.colors.qualitative.Set2)
    fig.update_traces(marker=dict(size=size,opacity=opacity))
    fig.update_layout(template='simple_white')
    fig.update_layout(legend= {'itemsizing': 'constant'})
    fig.write_html(filename+'.spatial_projection.html', auto_open=auto_open)
    return 


In [5]:
df = pd.read_pickle('../data/id_52.measurements.smoothed.r10000.pkl')
scattered_wsi(df,'cx','cy','area',1,1,True,'r10000')

In [7]:
# Plot the morphology profiles by clusterID3
features2cluster = ['area',
                    #'perimeter', 
                    #'solidity',
                    'eccentricity', 
                    'circularity', 
                    'mean_intensity', 
                    'cov_intensity'
                    ]
for filename in glob('../data_intensity/pkl/id_*.measurements.covd.pkl.intensityANDmorphology.csv.gz')[:1]:
    print(filename)
    df = pd.read_csv(filename)
    df['clusterID3'] = df['clusterID3'].add(1)
    fig = plt.figure(figsize=(15,8))
    fig.subplots_adjust(hspace=0.4, wspace=0.4)
    #plot covd clusters
    dfmelted = pd.DataFrame()
    for c in set(df["clusterID1"]):
        dfc = df[df["clusterID1"] == c][features2cluster]
        data = pd.melt(dfc)
        data['ID'] = 'id_'+str(c)
        dfmelted = dfmelted.append(data)
    ax = fig.add_subplot(1, 2, 1)
    ax.set(ylim=(0, 1))
    sns.lineplot(x="variable", y="value",hue='ID',
                 #style='ID',
                err_style = 'band',
                data=dfmelted, 
                ci='sd',
                #legend='full',
                markers=False, 
                legend=False).set_title('Feature mean value profiles by CovD-only cluster ID')
    #plot profile clusters
    dfmelted = pd.DataFrame()
    for c in set(df["clusterID3"]):
        dfc = df[df["clusterID3"] == c][features2cluster]
        data = pd.melt(dfc)
        data['ID'] = 'id_'+str(c)
        dfmelted = dfmelted.append(data)
    ax = fig.add_subplot(1, 2, 2)
    ax.set(ylim=(0, 1))
    ax.text(0.5, 0.7, str(df['clusterID3'].value_counts().to_frame()),fontsize=12, ha='center')
    sns.lineplot(x="variable", y="value",hue='ID',
                 #style='ID',
                err_style = 'band',
                data=dfmelted, 
                ci='sd',
                markers=False, 
                dashes=False).set_title('Feature mean value profiles by final cluster ID')
    ax.legend(loc='upper right')
    fig.savefig(filename+'.profiles.png') 
   

../data_intensity/pkl/id_45.measurements.covd.pkl.intensityANDmorphology.csv.gz
{0, 1, 2, 3}


In [14]:
for filename in glob('../data_intensity/pkl/id_52.measurements.covd.pkl.intensityANDmorphology.csv.gz')[:1]:
    print(filename)
    df = pd.read_csv(filename)

    df['clusterID3'] = df['clusterID3'].add(1)
    # set coloring feature
    df['ID'] = df.apply(lambda row: 'id_'+str(int(row.clusterID3)), axis=1)
    clusters = list(set(df['clusterID3'])) 
    for c in clusters:
        fdf = df['clusterID3'] == c
        scattered_wsi(df[fdf],'cx','cy','area',2,1,True,filename+str(c))


../data_intensity/pkl/id_52.measurements.covd.pkl.intensityANDmorphology.csv.gz


In [9]:
df.columns

Index(['fov_row', 'fov_col', 'cx', 'cy', 'area', 'perimeter', 'solidity',
       'eccentricity', 'circularity', 'mean_intensity', 'std_intensity',
       'cov_intensity', 'xi', 'yi', 'zi', 'curvature', 'clusterID1',
       'clusterID2', 'clusterID3', 'ID'],
      dtype='object')